In [1]:
%run "./preprocessing.ipynb"

C:\Users\SB\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Done !!


In [266]:
import warnings
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

tf.__version__
warnings.filterwarnings(action='ignore')

In [3]:
e_commerce_fc_data.head(1)

,참고초드,주문 날짜,주문 시간,고객사코드,품목수량,품목금액,입력자ID,권역구분,송화인주소1,송화인주소2,...,datetime,송화인터미널,수화인터미널,SHPR_ADDR_LATITUDE,SHPR_ADDR_LONGITUDE,SHPR_ADDR_HUBS,CNEE_ADDR_LATITUDE,CNEE_ADDR_LONGITUDE,CNEE_ADDR_HUBS,distance
2021-03-03 03:55:21,KX007,20210303,35521,90001441,1,20900.0,iNEXBatch,50.0,경기도,광주시,...,2021-03-03 03:55:21,경기도 광주시,경상남도 김해시,37.403,127.239,분당ASub,35.236,128.831,김해ASub,340.199185


In [238]:
np.random.seed(seed=42)    
random_number = np.random.randint(0, len(e_commerce_fc_data), 100000)

selected_data = e_commerce_fc_data.iloc[random_number, :][["datetime", "고객사코드", "distance"]]

In [239]:
### 출고량 별 세분화 
def set_fee(selected):
    code_fee_ = {
        100000 : 0,
        50000 : 20,
        30000 : 40,
        20000 : 60,
        10000 : 80,
        5000 : 100,
        3000 : 120,
        2000 : 140,
        1000 : 160,
        500 : 180,
        0 : 200
    }

    code_list = {
        90001302 : 1,
        90001542 : 1,
        90001341 : 1,
        90001541 : 1,
        90001441 : 1,
        90001602 : 1,
        90001443 : 1,
        90001582 : 81950,
        90001622 : 68074,
        90001521 : 58814,
        90001682 : 36071,
        90001702 : 30606,
        90001776 : 24318,
        90001705 : 23976,
        90001704 : 20317,
        90001768 : 12844,
        90001662 : 8126,
        90001683 : 5850,
        90001381 : 5438,
        90001703 : 2416,
        90001664 : 1904,
        90001502 : 133,
        90001765 : 48,
        90001842 : 44,
        90001562 : 10,
        90001774 : 7,
        90001802 : 6
    }

    ### 거리별  세분화
    distance_fee_ = {
        400 : 500,
        300 : 400,
        100 : 300,
        50 : 200,
        0 : 0
    }

    ### 시간별 세분화
    time_fee_ = {
        400 : [1, 2, 11, 12, 13],
        200 : [0, 3, 4, 10, 14],
        100 : [5, 6, 7, 8, 9, 15, 23],
        0 : [16, 17, 18, 19, 20, 21, 22]
    }

    make_code_fee_ = []
    make_distance_fee_ = []
    make_time_fee_ = []
    make_fee_ = []
    code_fee_key = list(code_fee_.keys())
    distance_fee_key = list(distance_fee_.keys())
    time_fee_key = list(time_fee_.keys())

    for i in range(len(selected)):
        target_code = selected["고객사코드"][i]
        target_distance = selected["distance"][i]
        target_day = selected["datetime"][i]


        ## code fee_ 
        code_value = code_list[target_code]

        if code_value == 1:
            set_code_fee = 0
            make_code_fee_.append(set_code_fee)
        else:
            for i in range(len(code_fee_key) -1 ):

                if (code_fee_key[i] > code_value) and ( code_fee_key[i+1] <= code_value) :
                    set_code_fee = code_fee_[code_fee_key[i+1]]
                    set_code_fee = set_code_fee + (code_fee_[code_fee_key[i+1]] - code_fee_[code_fee_key[i]]) * ( (code_value - code_fee_key[i+1]) / (code_fee_key[i] - code_fee_key[i+1]) )

                    if set_code_fee > 200:
                        set_code_fee = 200
                    make_code_fee_.append(int(set_code_fee))
                else:
                    pass

        ## distance fee_
        if target_distance >= distance_fee_key[0]:
            set_distance_fee = 500
            make_distance_fee_.append(set_distance_fee)
        else:
            for i in range(len(distance_fee_key) -1 ):
                if (distance_fee_key[i] > target_distance) and ( distance_fee_key[i+1] <= target_distance):
                    set_distance_fee = distance_fee_[distance_fee_key[i+1]]
                    set_distance_fee = set_distance_fee + (distance_fee_[distance_fee_key[i+1]] - distance_fee_[distance_fee_key[i]]) * (( target_distance - distance_fee_key[i+1]) / ( distance_fee_key[i] - distance_fee_key[i+1]) )

                    if set_distance_fee > 500:
                        set_distance_fee = 200

                    if set_distance_fee < 0:
                        set_distance_fee = 0
                    make_distance_fee_.append(int(set_distance_fee))
                else:
                    pass

        ## day fee_

        day_value = target_day.hour

        for i in time_fee_key:
            if day_value in time_fee_[i]:
                set_day_fee = i
                make_time_fee_.append(set_day_fee)            
            else:
                pass

    for i in range(len(selected)):
        make_fee_.append(
            2100 
            + make_code_fee_[i]
            + make_distance_fee_[i]
            + make_time_fee_[i]
        )


    selected["result"] = make_fee_
    selected["code"] = make_code_fee_
    
    return selected

In [240]:
set_fee_df = set_fee(selected_data)

In [241]:
set_fee_df

,datetime,고객사코드,distance,result,code
2021-03-15 21:47:56,2021-03-15 21:47:56,90001302,70.990083,2258,0
2021-05-07 05:39:21,2021-05-07 05:39:21,90001541,63.136312,2373,0
2021-03-16 18:27:14,2021-03-16 18:27:14,90001541,268.493797,2315,0
2021-06-16 07:27:41,2021-06-16 07:27:41,90001768,91.661964,2401,85
2021-03-28 14:09:05,2021-03-28 14:09:05,90001541,62.113800,2475,0
...,...,...,...,...,...
2021-05-30 09:04:42,2021-05-30 09:04:42,90001582,141.974597,2511,32
2021-05-28 14:24:19,2021-05-28 14:24:19,90001443,357.925071,2642,0
2021-06-02 12:21:06,2021-06-02 12:21:06,90001302,24.181382,2500,0
2021-04-16 03:00:18,2021-04-16 03:00:18,90001302,82.054098,2435,0


In [243]:
set_fee_df["time"] = 0
for i in range(len(set_fee_df)):

    set_fee_df["time"][i] = set_fee_df["datetime"][i].hour

In [248]:
train_x = set_fee_df[["distance", "time", "code"]].iloc[:80000, :]
train_y = set_fee_df[["result"]].iloc[:80000, :]

test_x = set_fee_df[["distance", "time", "code"]].iloc[80000:, :]
test_y = set_fee_df[["result"]].iloc[80000:, :]

In [249]:
batch_size = 100
model = Sequential()
model.add(Dense(units=64, activation="relu", input_shape=(batch_size, 3, )))
model.add(Dense(32))
model.add(Dense(1))

model.compile(loss="mean_squared_error", optimizer="adam")

In [251]:
model.fit(train_x, train_y, batch_size = batch_size, epochs=2000, verbose=1)

Epoch 1/2000
800/800 [==============================] - 1s 1ms/step - loss: 24274.4863
Epoch 2/2000
800/800 [==============================] - 1s 1ms/step - loss: 23626.5879
Epoch 3/2000
800/800 [==============================] - 1s 1ms/step - loss: 23235.4531
Epoch 4/2000
800/800 [==============================] - 1s 1ms/step - loss: 22780.7852
Epoch 5/2000
800/800 [==============================] - 1s 1ms/step - loss: 22609.9004
Epoch 6/2000
800/800 [==============================] - 1s 1ms/step - loss: 22225.7930
Epoch 7/2000
800/800 [==============================] - 1s 1ms/step - loss: 21658.4746
Epoch 8/2000
800/800 [==============================] - 1s 1ms/step - loss: 21291.7715
Epoch 9/2000
800/800 [==============================] - 1s 1ms/step - loss: 20604.4590
Epoch 10/2000
800/800 [==============================] - 1s 1ms/step - loss: 20344.7773
Epoch 11/2000
800/800 [==============================] - 1s 1ms/step - loss: 20314.6680
Epoch 12/2000
800/800 [==================

800/800 [==============================] - 1s 1ms/step - loss: 11010.9561
Epoch 95/2000
800/800 [==============================] - 1s 1ms/step - loss: 10883.6836
Epoch 96/2000
800/800 [==============================] - 1s 1ms/step - loss: 10627.6289
Epoch 97/2000
800/800 [==============================] - 1s 999us/step - loss: 10609.2549
Epoch 98/2000
800/800 [==============================] - 1s 1ms/step - loss: 10508.2285
Epoch 99/2000
800/800 [==============================] - 1s 1ms/step - loss: 10733.3145
Epoch 100/2000
800/800 [==============================] - 1s 1ms/step - loss: 10337.4619
Epoch 101/2000
800/800 [==============================] - 1s 1ms/step - loss: 10368.8115
Epoch 102/2000
800/800 [==============================] - 1s 1ms/step - loss: 10367.8018
Epoch 103/2000
800/800 [==============================] - 1s 980us/step - loss: 10201.2988
Epoch 104/2000
800/800 [==============================] - 1s 1ms/step - loss: 10260.1699
Epoch 105/2000
800/800 [=============

800/800 [==============================] - 1s 1ms/step - loss: 3583.0481
Epoch 187/2000
800/800 [==============================] - 1s 1ms/step - loss: 3344.0503
Epoch 188/2000
800/800 [==============================] - 1s 1ms/step - loss: 3216.1189
Epoch 189/2000
800/800 [==============================] - 1s 1ms/step - loss: 3322.1157
Epoch 190/2000
800/800 [==============================] - 1s 1ms/step - loss: 3299.9670
Epoch 191/2000
800/800 [==============================] - 1s 1ms/step - loss: 3151.8047
Epoch 192/2000
800/800 [==============================] - 1s 1ms/step - loss: 3190.0359
Epoch 193/2000
800/800 [==============================] - 1s 989us/step - loss: 3180.4937
Epoch 194/2000
800/800 [==============================] - 1s 1ms/step - loss: 3097.3606
Epoch 195/2000
800/800 [==============================] - 1s 1ms/step - loss: 3134.6191
Epoch 196/2000
800/800 [==============================] - 1s 1ms/step - loss: 3221.6199
Epoch 197/2000
800/800 [=====================

Epoch 279/2000
800/800 [==============================] - 1s 1ms/step - loss: 2762.6917
Epoch 280/2000
800/800 [==============================] - 1s 1ms/step - loss: 2415.3635
Epoch 281/2000
800/800 [==============================] - 1s 1ms/step - loss: 2496.4817
Epoch 282/2000
800/800 [==============================] - 1s 1ms/step - loss: 2499.6548
Epoch 283/2000
800/800 [==============================] - 1s 1ms/step - loss: 2473.1423
Epoch 284/2000
800/800 [==============================] - 1s 1ms/step - loss: 2432.2783
Epoch 285/2000
800/800 [==============================] - 1s 1ms/step - loss: 2634.9939
Epoch 286/2000
800/800 [==============================] - 1s 2ms/step - loss: 2379.3914
Epoch 287/2000
800/800 [==============================] - 1s 1ms/step - loss: 2522.4973
Epoch 288/2000
800/800 [==============================] - 1s 1ms/step - loss: 2444.0242
Epoch 289/2000
800/800 [==============================] - 1s 1ms/step - loss: 2544.4736
Epoch 290/2000
800/800 [========

800/800 [==============================] - 1s 1ms/step - loss: 1821.8118
Epoch 373/2000
800/800 [==============================] - 1s 1ms/step - loss: 1880.0140
Epoch 374/2000
800/800 [==============================] - 1s 1ms/step - loss: 1740.7650
Epoch 375/2000
800/800 [==============================] - 1s 1ms/step - loss: 1704.4952
Epoch 376/2000
800/800 [==============================] - 1s 1ms/step - loss: 2161.2273
Epoch 377/2000
800/800 [==============================] - 1s 1ms/step - loss: 1588.6847
Epoch 378/2000
800/800 [==============================] - 1s 1ms/step - loss: 1830.9550
Epoch 379/2000
800/800 [==============================] - 1s 1ms/step - loss: 1738.9774
Epoch 380/2000
800/800 [==============================] - 1s 1ms/step - loss: 1855.1462
Epoch 381/2000
800/800 [==============================] - 1s 1ms/step - loss: 1890.6444
Epoch 382/2000
800/800 [==============================] - 1s 1ms/step - loss: 1786.0116
Epoch 383/2000
800/800 [=======================

...

800/800 [==============================] - 1s 1ms/step - loss: 881.8583
Epoch 1579/2000
800/800 [==============================] - 1s 1ms/step - loss: 899.3305
Epoch 1580/2000
800/800 [==============================] - 1s 1ms/step - loss: 809.2562
Epoch 1581/2000
800/800 [==============================] - 1s 1ms/step - loss: 903.3179
Epoch 1582/2000
800/800 [==============================] - 1s 1ms/step - loss: 891.5419
Epoch 1583/2000
800/800 [==============================] - 1s 1ms/step - loss: 824.9263
Epoch 1584/2000
800/800 [==============================] - 1s 1ms/step - loss: 940.0221
Epoch 1585/2000
800/800 [==============================] - 1s 2ms/step - loss: 818.9872
Epoch 1586/2000
800/800 [==============================] - 1s 1ms/step - loss: 893.8174
Epoch 1587/2000
800/800 [==============================] - 1s 1ms/step - loss: 814.6135
Epoch 1588/2000
800/800 [==============================] - 1s 1ms/step - loss: 915.5300
Epoch 1589/2000
800/800 [=======================

Epoch 1671/2000
800/800 [==============================] - 1s 1ms/step - loss: 796.1530
Epoch 1672/2000
800/800 [==============================] - 1s 1ms/step - loss: 856.4421
Epoch 1673/2000
800/800 [==============================] - 1s 1ms/step - loss: 833.1860
Epoch 1674/2000
800/800 [==============================] - 1s 1ms/step - loss: 876.0218
Epoch 1675/2000
800/800 [==============================] - 1s 1ms/step - loss: 776.9796
Epoch 1676/2000
800/800 [==============================] - 1s 1ms/step - loss: 885.3169
Epoch 1677/2000
800/800 [==============================] - 1s 1ms/step - loss: 887.6949
Epoch 1678/2000
800/800 [==============================] - 1s 1ms/step - loss: 840.0224
Epoch 1679/2000
800/800 [==============================] - 1s 1ms/step - loss: 860.4373
Epoch 1680/2000
800/800 [==============================] - 1s 1ms/step - loss: 832.6531
Epoch 1681/2000
800/800 [==============================] - 1s 1ms/step - loss: 812.1824
Epoch 1682/2000
800/800 [=======

Epoch 1764/2000
800/800 [==============================] - 1s 1ms/step - loss: 924.6712
Epoch 1765/2000
800/800 [==============================] - 1s 1ms/step - loss: 825.8127
Epoch 1766/2000
800/800 [==============================] - 1s 1ms/step - loss: 784.3649
Epoch 1767/2000
800/800 [==============================] - 1s 1ms/step - loss: 812.8475
Epoch 1768/2000
800/800 [==============================] - 1s 1ms/step - loss: 832.5380
Epoch 1769/2000
800/800 [==============================] - 1s 1ms/step - loss: 857.9498
Epoch 1770/2000
800/800 [==============================] - 1s 1ms/step - loss: 790.0574
Epoch 1771/2000
800/800 [==============================] - 1s 1ms/step - loss: 836.1205
Epoch 1772/2000
800/800 [==============================] - 1s 1ms/step - loss: 887.4719
Epoch 1773/2000
800/800 [==============================] - 1s 1ms/step - loss: 832.2048
Epoch 1774/2000
800/800 [==============================] - 1s 1ms/step - loss: 829.6201
Epoch 1775/2000
800/800 [=======

800/800 [==============================] - 1s 1ms/step - loss: 901.4831
Epoch 1858/2000
800/800 [==============================] - 1s 1ms/step - loss: 853.9948
Epoch 1859/2000
800/800 [==============================] - 1s 1ms/step - loss: 830.3729
Epoch 1860/2000
800/800 [==============================] - 1s 1ms/step - loss: 769.7922
Epoch 1861/2000
800/800 [==============================] - 1s 1ms/step - loss: 883.3245
Epoch 1862/2000
800/800 [==============================] - 1s 1ms/step - loss: 788.1539
Epoch 1863/2000
800/800 [==============================] - 1s 1ms/step - loss: 878.7503
Epoch 1864/2000
800/800 [==============================] - 1s 1ms/step - loss: 816.5627
Epoch 1865/2000
800/800 [==============================] - 1s 1ms/step - loss: 775.8245
Epoch 1866/2000
800/800 [==============================] - 1s 1ms/step - loss: 815.3455
Epoch 1867/2000
800/800 [==============================] - 1s 1ms/step - loss: 862.9477
Epoch 1868/2000
800/800 [=======================

800/800 [==============================] - 1s 1ms/step - loss: 822.6224
Epoch 1950/2000
800/800 [==============================] - 1s 1ms/step - loss: 896.7454
Epoch 1951/2000
800/800 [==============================] - 1s 1ms/step - loss: 836.1763
Epoch 1952/2000
800/800 [==============================] - 1s 1ms/step - loss: 776.3454
Epoch 1953/2000
800/800 [==============================] - 1s 1ms/step - loss: 838.4178
Epoch 1954/2000
800/800 [==============================] - 1s 1ms/step - loss: 783.8054
Epoch 1955/2000
800/800 [==============================] - 1s 1ms/step - loss: 810.2004
Epoch 1956/2000
800/800 [==============================] - 1s 1ms/step - loss: 802.7966
Epoch 1957/2000
800/800 [==============================] - 1s 1ms/step - loss: 803.0776
Epoch 1958/2000
800/800 [==============================] - 1s 1ms/step - loss: 788.7025
Epoch 1959/2000
800/800 [==============================] - 1s 1ms/step - loss: 884.3706
Epoch 1960/2000
800/800 [=======================

In [254]:
pred = model.predict(test_x)

In [255]:
mean_absolute_error(pred, test_y), mean_squared_error(pred, test_y)

(18.98571279296875, 782.1542864532888)

In [259]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

svr = SVR()
rfg = RandomForestRegressor()
abr = AdaBoostRegressor()
dtr = DecisionTreeRegressor()

In [260]:
svr.fit(train_x, train_y)
rfg.fit(train_x, train_y)
abr.fit(train_x, train_y)
dtr.fit(train_x, train_y)

svr_pred = svr.predict(test_x)
rfg_pred = rfg.predict(test_x)
abr_pred = abr.predict(test_x)
dtr_pred = dtr.predict(test_x)

In [261]:
print(mean_absolute_error(svr_pred, test_y))
print(mean_absolute_error(rfg_pred, test_y))
print(mean_absolute_error(abr_pred, test_y))
print(mean_absolute_error(dtr_pred, test_y))

124.30039648771027
0.21907400000000388
92.09909601332713
0.22925


In [264]:
print(mean_squared_error(svr_pred, test_y))
print(mean_squared_error(rfg_pred, test_y))
print(mean_squared_error(abr_pred, test_y))
print(mean_squared_error(dtr_pred, test_y))

22589.670690583385
3.4796300100000024
10631.326397881196
5.20485


In [267]:
with open("./model/set_fee_mode.pickle", "wb") as f:
    pickle.dump(rfg, f, pickle.HIGHEST_PROTOCOL)
    